In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import scipy.stats
import datetime as dt
import yfinance as yf

import fin_func as ff

# <font face="gotham" color="purple"> Value at Risk</font>

Though returns are hardly normal distribution, they can be seen as a linear combination of standard normal distribution
$$
r(t) = \mu+ \sigma*N(0, 1)
$$

With this knowledge, we can characterize losses in price units, for instance, 'with 5% degree of confidence your portfolio could lose up to USD23000 next week'.

$$
\text{Prob}(\delta \pi\leq -\text{VaR}) = 1-\text{Confi. Lvl.}
$$
where $\delta \pi$ means the change of portfolio.

Here's how you translate the time interval
$$
\sigma_{\text{n-day}} = \sigma_{\text{daily}}\sqrt{n}\\
\mu_{\text{n-day}} = \mu_{\text{daily}}n
$$

This is the formula of $\text{VaR}$
$$
\text{VaR} = \Delta S [\underbrace{\mu \delta t}_{\text{from }  \mu_{\text{daily}}n} -\underbrace{\sigma \sqrt{\delta t}}_{\text{from }\sigma_{\text{daily}}\sqrt{n}}F^{-1}(1-C)]
$$
where $F^{-1}$ is the inverse cumulative distribution function.

In [89]:
def download_data(stocks, dropna, start_date, end_date, column):
    # dropna decides if you want to drop all NaN values, column for instance ['Close']
    try:
        if isinstance(stocks, list):
            stock_data = {}
            for stock in stocks:
                ticker = yf.Ticker(stock)
                stock_data[stock]=ticker.history(start=start_date, end=end_date)
                stock_data[stock] = stock_data[stock][column]
            if dropna == True:
                df = pd.DataFrame(stock_data).dropna()
            else:
                df = pd.DataFrame(stock_data)
        return df
    except:
        print('Tickers should be contained in a list, for instance, ["LULU"]')

def calculate_var(porfolio, confi, mu, sigma):
    F_inv = sp.stats.norm.ppf(1 - confi)
    var = porfolio * (mu - sigma*F_inv) # if the data is daily, and this calculates the VaR tomorrow
    print('VaR tomorrow under {:.2f} significance level is ${:.2f}'.format((1-confi), var))
    return var

def calculate_var_n(porfolio, confi, mu, sigma, n):
    F_inv = sp.stats.norm.ppf(1 - confi)
    var = porfolio * (mu *n - sigma*np.sqrt(n)*F_inv) # if the data is daily, and this calculates the VaR tomorrow
    print('VaR in {} days under {:.2f} significance level is ${:.2f}'.format(n, (1-confi), var))
    return var

if __name__ == '__main__':
    df['return'] = (np.log(df['TSLA']) - np.log(df['TSLA'].shift())).dropna()
    mu = df['return'].mean()
    sigma = df['return'].std()
    porfolio = 1e6
    confi = .95
    
    df=download_data(stocks=['TSLA'], dropna=True, start_date='2015-01-01', 
                     end_date=dt.datetime.today(), column='Close')
    VaR = calculate_var(porfolio, confi, mu, sigma)
    VaR_ = calculate_var_n(porfolio, confi, mu, sigma, 2)
    

VaR tomorrow under 0.05 significance level is $59675.94
VaR in 2 days under 0.05 significance level is $84985.07


## <font face="gotham" color="purple">  Monte Carlo Approach</font>

We will simulate stock prices with 
$$
S(t)=S(0) e^{\left(\mu-\frac{1}{2} \sigma^2\right) t+\sigma \sqrt{t}  N(0, 1)}
$$

In [118]:
class VaR_MC:
    def __init__(self, S, mu, sigma, confi, n, iterations):
        self.S = S
        self.mu = mu
        self.sigma = sigma
        self.confi = confi
        self.n = n
        self.iterations = iterations
        
    def simulation(self):
        rand_W = np.random.normal(0, 1, [1, self.iterations]) # size 1 by x, this will give a 2-d array
        stock_price = self.S * np.exp((self.mu - .5*self.sigma**2)*self.n + self.sigma * np.sqrt(self.n) * rand_W)
        stock_price = np.sort(stock_price)
        percentile = np.percentile(stock_price, (1-self.confi)*100)                            
        return self.S - percentile

if __name__ == '__main__':
    df['return'] = (np.log(df['TSLA']) - np.log(df['TSLA'].shift())).dropna()
    mu = df['return'].mean()
    sigma = df['return'].std()
    model = VaR_MC(1e6, mu, sigma, .95, 100, 1000)
    var = model.simulation()
var

401101.9170862766

In [93]:
var

415237.6243181953